**Installation**

Update and Upgrade On Linux System by Using Following Commands: 
```
Command$1 sudo apt update && sudo apt upgrade -y
Command$2 sudo apt install gcc
Command$3 sudo apt install cmake
Command$4 sudo apt install build-essential
Command$5 sudo apt install libfftw3-dev OR
Command$6 sudo apt-get install -y libfftw3-dev
```

**Install Chimera**


Move the file to desired folder: In that folder open terminal
```
Command$1 ls (to check the name of setup file)
Command$2 chmod +x chimera-1.17.1-linux_x86_64.bin
Command$3 sudo ./chimera-1.17.1-linux_x86_64.bin
```

**Install VMD**

Download Link for [VMD](https://www.ks.uiuc.edu/Development/Download/download.cgi?PackageName=VMD)
```
1. Save the .tar.gz file in working folder
2. open the extracted folder and run command$: './configure' by opening the termianl  
3. Open the folder src in terminal and run the command$: sudo make install
4. type 'vmd' in termial and program should run. 
```

Download link for [AutoDock-Vina](https://vina.scripps.edu/downloads/)
```
1. First extract the file and move the file on particular directory
2. Run through Chimera by setting the location on Tool/AutoDock-Vina Section 
```

**GROMACS-Compilation Wiith CUDA**

Download Link for [CUDA](https://developer.nvidia.com/cuda-downloads)


**Installation Steps:** Follow the each steps by copying and running the commands 


Download Link for [Gromacs](https://manual.gromacs.org/current/download.html)


**Installation Steps:**
1. Command$1: tar xfz gromacs-2020.2.tar.gz

2. Command$2: cd gromacs-2020.2		

3. Command$3: mkdir build		

4. Command$4: cd build			

5. Command$5: cmake .. -DGMX_GPU=CUDA -DCUDA_TOOLKIT_ROOT_DIR=/usr/local/cuda	

6. Command$6: make				

7. Command$7: make check	

8. Command$8: sudo make install		

9. Command$9: source /usr/local/gromacs/bin/GMXRC

**---Preparation of Ligand and Protein Using Chimera----**
1. Open the ligand.pdb file on Chimera and add H than save it as LIG.mols
2. Open the protein.pdb file on the Chimera add H and ions and save as REC.pdb
3. Open the LIG.mol2 file and in the second line correction to be made in LIG.mol2 

4. Open LIG.mol2 by using gedit command or simplyopening file in any text editor:
4.1   "@<TRIPOS>MOLECULE" make sure this is the first line in file delete the header and empty space
4.2.  "@<TRIPOS>MOLECULE” there will be name after this line maybe xxx.pdb or ****** or anything else
	change it to LIG 
    
5. bond orders "@<TRIPOS>BOND" will be arranged differently in each file arrange them in specific order to avoid errors by using following script below: `perl sort_mol2_bonds.pl LIG.mol2 LIG.mol2` script to order the bonds
    
**Create Topology**___________________
    
1. Go to [SwissParam](http://www.swissparam.ch/) and upload the 'Lig.mol2 file' 
2. Download the .zip folder
3. Again open the best pose ligand with the receptor .pdb file, delete ligand, Perform DockPrep of protein as save it as .pdb file as 'REC.pdb'
4. Make a working Folder for Gromacs, copy contents of the downloaded zip file into this folder, copy the DockPrep 'rec.pdb' in to working folder
5. Copy all the .mdp files into this working folder
6. Open the terminal in this working folder and proceed with Gromacs.

**---------Running GROMACS-----------:**
source /usr/local/gromacs/bin/GMXRC	(If Gromacs is manually compiled / not for dirty install)  
```
Command$1: gmx pdb2gmx -f REC.pdb -ignh
> 8 (CHARMM27)
> 1 (TIP3P)
Command$2: gmx editconf -f LIG.pdb -o LIG.gro
Command$: gedit conf.gro LIG.gro
> (Copy content from 3rd line of lig.gro to the conf.gro file up to the 2nd last line)
> (Check the column number from where the lig.gro data ends (x) in conf.gro and replace the value in 2nd line by x-3)
> (Open file in chimera to check ligand and receptor)
```



**-----EDIT THE FOLLOWING in topol.top -----**
```
gedit topol.top
(add 
; Include ligand topology
#include "LIG.itp"

below:
Include forcefield parameters
#include "amberGS.ff/forcefield.itp")

AT THE BOTTOM OF THE SAME FILE PERFORM FOLLOWING CHANGES
(add `LIG 1` align exactly below: Protein_chain_E     1)
```

**-----EDIT THE FOLLOWING in lig.itp -----**
```
gedit lig.itp
[ moleculetype ]
; Name nrexcl
lig_gmx2 3
TO
[ moleculetype ]
; Name nrexcl
LIG 3 (in certain cases this will already be LIG 3 so for such case no change is needed)
```

**Solvent box and Energy Minimization:**
```
Command$3: gmx editconf -f conf.gro -d 1.0 -bt triclinic -o box.gro
Command$4: gmx solvate -cp box.gro -cs spc216.gro -p topol.top -o box_sol.gro
Command$5: gmx grompp -f ions.mdp -c box_sol.gro -p topol.top -o ION.tpr`
Command$6: gmx genion -s ION.tpr -p topol.top -conc 0.1 -neutral -o box_sol_ion.gro

> Select group : 15

Command$7: gmx grompp -f EM.mdp -c box_sol_ion.gro -p topol.top -o EM.tpr
Command$9: gmx mdrun -v -deffnm EM
```
    

    
**Now make index files**
```
Command$10: gmx make_ndx -f LIG.gro -o index_LIG.ndx
Types:
> 0 & ! a H*
> q
Command$11: gmx genrestr -f LIG.gro -n index_LIG.ndx -o posre_LIG.itp -fc 1000 1000 1000
> Select group "3"
```
    
    
**Now, open `topol.top` file**

at the end of the document 

> after
```
"; Include Position restraint file
#ifdef POSRES
#include "posre.itp"
#endif
```
> "Here": add this 
```
; Ligand position restraints
#ifdef POSRES
#include "posre_LIG.itp"
#endif
```

**Again, Make other Index file for System**
```
Command$12: gmx make_ndx -f EM.gro -o index.ndx
> 1 | 13
> q  
```

**-----[NVT]-----**:
gedit NVT.mdp (This file is already modified)
```
Command$1: gmx grompp -f NVT.mdp -c EM.gro -r EM.gro -p topol.top -n index.ndx -o NVT.tpr
Command$2: gmx mdrun -deffnm NVT
```


**-----[NPT]-----**:
gedit NPT.mdp (This file is already modified)
```
Command$3: gmx grompp -f NPT.mdp -c NVT.gro -r NVT.gro -p topol.top -n index.ndx -maxwarn 2 -o NPT.tpr
Command$4: gmx mdrun -deffnm NPT
```

```
In summary, the NVT ensemble is used to simulate systems where the number of particles, volume and temperature are kept constant during the simulation, while the NPT ensemble is used to simulate systems where the number of particles, volume, and pressure are constant.
```

**-----[FINAL MD RUN/PRODUCTION]-----**
gedit MD.mdp (Change MD RUN TIME as per your need)
```
Command$:1 gmx grompp -f MD.mdp -c NPT.gro -t NPT.cpt -p topol.top -n index.ndx -maxwarn 2 -o MD.tpr
Command$:2 gmx mdrun -deffnm MD
```


**----[Recentering and Rewrapping Coordinates]----**
```
Command$:1 gmx trjconv -s MD.tpr -f MD.xtc -o MD_center.xtc -center -pbc mol -ur compact
```
+ Choose "Protein" for centering and "System" for output.

+ To extract the first frame (t = 0 ns) of the trajectory, use trjconv -dump with the recentered trajectory:
```
Command$:2 gmx trjconv -s MD.tpr -f MD_center.xtc -o start.pdb -dump 0
```

**Plotting using grace:**

**------RMSD Calculations-----**
```
Command$: gmx rms -s MD.tpr -f MD_center.xtc -o rmsd.xvg
Command$: gmx rms -s MD.tpr -f MD_center.xtc -o rmsd.xvg -tu ns
4
13
Select appropritate 2 options one by one and then open the output files in Grace) Select Backbone and then LIG
Command$: xmgrace rmsd.xvg
```

**------RMSF Calculations-----**
```
Command$: gmx rmsf -s MD.tpr -f MD_center.xtc -o rmsf.xvg
4
Select appropritate Backbone open the output files in Grace
Command$: xmgrace output.xvg
```
**-----------H-bonds-------------**
```
Command$: gmx hbond -s MD.tpr -f MD_center.xtc -num hb.xvg
Command$: gmx hbond -s MD.tpr -f MD_center.xtc -num hb.xvg -tu ns
1
13
Command$: xmgrace hb.xvg
```

**----------Gyration Radius----------**
```
Command$: gmx gyrate -s MD.tpr -f MD_center.xtc -o gyrate1.xvg
#Choose the group of your choice 
Command$: xmgrace gyrate1.xvg
```

**----------ENERGY Calculations----------**
```
Command$: gmx energy -f MD.edr -o energy1.xvg
#Choose the option of your choice
Command$: xmgrace -nxy energy1.xvg
```